# Initialization

In [14]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [15]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [16]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

events_train = pd.read_parquet("events_train.par")
events_test = pd.read_parquet("events_test.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [17]:
users_train = events_train["user_id"].drop_duplicates().to_frame().reset_index(drop=True)
users_test = events_test["user_id"].drop_duplicates().to_frame().reset_index(drop=True)

common_users = users_train.merge(users_test,on="user_id",how="outer",indicator="indic")
common_users

,user_id,indic
0,1000000,left_only
1,1000001,left_only
2,1000002,left_only
3,1000003,both
4,1000004,left_only
...,...,...
430580,1429834,right_only
430581,1429875,right_only
430582,1430053,right_only
430583,1430311,right_only


In [18]:
cold_users = common_users[common_users["indic"] == "right_only"]["user_id"]

print(len(cold_users)) 

2365


In [37]:
cold_users

428220    1000153
428221    1000325
428222    1000504
428223    1000712
428224    1000806
           ...   
430580    1429834
430581    1429875
430582    1430053
430583    1430311
430584    1430500
Name: user_id, Length: 2365, dtype: int64

In [19]:
events_train

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1000000,5350,2016-03-18,2016-04-03,True,4,False,2016-03-01
1,1000000,6748,2016-04-16,2016-04-30,True,5,False,2016-04-01
2,1000000,17675462,2016-07-06,2016-07-15,True,5,False,2016-07-01
3,1000000,25494343,2016-06-10,2016-07-06,True,4,False,2016-06-01
4,1000000,17851885,2016-08-01,2016-08-09,True,4,False,2016-08-01
...,...,...,...,...,...,...,...,...
11751081,1430584,7896527,2016-05-18,2016-06-03,True,4,True,2016-05-01
11751082,1430584,29056083,2016-08-01,2016-08-03,True,3,True,2016-08-01
11751083,1430584,6614960,2015-11-02,2015-12-25,True,3,False,2015-11-01
11751084,1430584,3153910,2014-06-11,2014-07-01,True,5,False,2014-06-01


In [20]:
# топ-100 наиболее популярных книг

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity

,item_id,users,avg_rating
0,1,8728,4.796059
1,2,9528,4.685873
2,3,15139,4.706057
3,5,11890,4.770143
4,6,10679,4.764772
...,...,...,...
38909,35783057,3,5.000000
38910,35818330,49,4.489796
38911,35828620,1,4.000000
38912,35852112,2,5.000000


In [21]:
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]
item_popularity


,item_id,users,avg_rating,popularity_weighted
0,1,8728,4.796059,41860.0
1,2,9528,4.685873,44647.0
2,3,15139,4.706057,71245.0
3,5,11890,4.770143,56717.0
4,6,10679,4.764772,50883.0
...,...,...,...,...
38909,35783057,3,5.000000,15.0
38910,35818330,49,4.489796,220.0
38911,35828620,1,4.000000,4.0
38912,35852112,2,5.000000,10.0


In [24]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted",ascending=False).reset_index(drop=True)
item_popularity


,item_id,users,avg_rating,popularity_weighted
0,22557272,40690,3.788965,154173.0
1,29056083,25785,3.801784,98029.0
2,18007564,20207,4.321275,87320.0
3,18143977,19462,4.290669,83505.0
4,16096824,16770,4.301014,72128.0
...,...,...,...,...
38909,935055,1,1.000000,1.0
38910,454062,1,1.000000,1.0
38911,863687,1,1.000000,1.0
38912,13010310,1,1.000000,1.0


In [34]:
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"]>=4].head(100).reset_index(drop=True)
top_k_pop_items

,item_id,users,avg_rating,popularity_weighted
0,18007564,20207,4.321275,87320.0
1,18143977,19462,4.290669,83505.0
2,16096824,16770,4.301014,72128.0
3,3,15139,4.706057,71245.0
4,38447,14611,4.232770,61845.0
...,...,...,...,...
95,2767052,4361,4.413437,19247.0
96,18293427,4674,4.092640,19129.0
97,3636,4667,4.098564,19128.0
98,18966819,4361,4.374914,19079.0


In [35]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 10):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
...,...,...,...,...,...,...,...,...
95,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2008,4361,4.413437,19247.0,"{'Young Adult': 30042, 'Fiction': 16754, 'Scie..."
96,18293427,Gabrielle Zevin,The Storied Life of A.J. Fikry,2014,4674,4.092640,19129.0,"{'Fiction': 3795, 'Contemporary': 1100, 'Writi..."
97,3636,Lois Lowry,"The Giver (The Giver, #1)",2006,4667,4.098564,19128.0,"{'Young Adult': 10993, 'Fiction': 9045, 'Class..."
98,18966819,Pierce Brown,"Golden Son (Red Rising, #2)",2015,4361,4.374914,19079.0,"{'Science Fiction': 2613, 'Fantasy': 1372, 'Sc..."


In [36]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам?
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")
cold_users_events_with_recs

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,1000153,29923707,2017-10-08,2017-10-13,True,4,False,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,1000153,31450852,2017-09-09,2017-09-17,True,5,False,2017-09-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,1000153,30025336,2017-09-27,2017-10-07,True,4,False,2017-09-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,1000153,18806259,2017-08-08,2017-08-12,True,3,False,2017-08-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,1000153,13262783,2017-08-12,2017-08-14,True,5,False,2017-08-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9667,1430500,28257707,2017-10-05,2017-10-07,True,4,False,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
9668,1430500,11387515,2017-10-15,2017-10-15,True,5,True,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
9669,1430500,25855506,2017-10-07,2017-10-07,True,5,False,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
9670,1430500,30754980,2017-10-07,2017-10-07,True,5,False,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [38]:
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_items_no_avg_rating_idx

0       True
1       True
2       True
3       True
4       True
        ... 
9667    True
9668    True
9669    True
9670    True
9671    True
Name: avg_rating, Length: 9672, dtype: bool

In [39]:
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]
cold_user_recs

,user_id,item_id,rating,avg_rating
14,1000504,1885,5,4.316316
16,1000712,13496,5,4.440779
21,1001508,1885,5,4.316316
24,1002222,15839976,4,4.150180
26,1002222,18966819,5,4.374914
...,...,...,...,...
9636,1429316,9361589,2,4.085858
9638,1429316,18584855,4,4.071619
9642,1429720,9969571,4,4.290044
9644,1429798,11235712,5,4.179189


In [42]:
# Верный ответ — 0,8023 (без округления). Книги в top_k_pop_items настолько популярны, что большая часть «холодных» пользователей их читала! 
1-1912/9672

0.8023159636062862

In [44]:
# посчитаем метрики рекомендаций
# Посчитайте метрики rmse и mae для полученных рекомендаций.
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2))

# Верный ответ — 0,62. В среднем оценка рекомендации отклоняется от истинной на такую величину.
#  Для пятибалльной шкалы отклонение невысокое, но и метрика посчитана по популярным книгам с высокими оценками. 

0.78 0.62


In [45]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

# «Холодных» пользователей без каких-либо релевантных рекомендаций — 57%,
#  то есть пересечение между оценёнными книгами и рекомендациями есть только у 43%, 
# по ним же и получено значение MAE-метрики. При этом среднее покрытие — 46%. 
# Это значит, что большая часть «холодных» пользователей не получила никаких релевантных рекомендаций, 
# а оставшаяся часть имеет пересечения только по 46% книг. 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

# === Базовые подходы: коллаборативная фильтрация

# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков